In [1]:
import weightipy as wp

In [2]:
import pandas as pd

In [108]:
ms=pd.read_excel(r'C:\Users\User\Downloads\ac1.xlsx')
ms['mla_2018'].replace('Independent candidateÂ', 'IND', inplace=True)

In [155]:
retro=pd.read_excel(r'E:\Manish PY\2018Retro.xlsx')

In [109]:
caste=pd.read_excel(r'E:\Manish PY\mpcastefinal.xlsx')

In [ ]:
#filter the set of castes which are more than 3% in universe and more than 5 samples
#reset caste dict to reflect set of caste and rest as 'others'
#make a column in ms 'caste map' to reflect castes in set and rest as 'others'
#run caste weights


In [ ]:
#retro mapping
#decide poles- above 6%
#check for samples and make set of parties (samples>5)
#make retro_dict by including set and 'others' as rest
#make 'party mapped' in ms to have parties from set and rest as 'others'
#Run retro weights

In [ ]:
aclist=ms['ac_no'].unique()
for i in range(len(aclist)):
    #ms
    try:
        ac=ms[ms['ac_no']==aclist[i]]
        accaste=caste[caste['ac_no']==aclist[i]]
        acretro=retro[(retro['Key']=='2018 AE') & (retro['AC No']==aclist[i])]


        #####################################################CASTE DICT FOR WEIGHING AND CASTE MAP COLUMN
        #1. finding castes with more than 5 samples
        l= ac['caste_clean'].value_counts()
        l=l.reset_index(name='count')
        l=l[l['count']>5]
        s1=set(l['caste_clean'])

        #2. FINDING CASTE WITH MORE THAN 3% IN UNIVERSE
        l= accaste[accaste['pop']>0.03]
        s2=set(l['clean caste'])

        #3. Finsing intersection
        s3=s1.intersection(s2)

        #4. reseting caste dict of AC
        accaste=caste[caste['ac_no']==aclist[i]]
        accaste=accaste[['clean caste','pop']]
        accaste=accaste[accaste['clean caste'].isin(s3)].reset_index()
        index_of_others = accaste.index[accaste['clean caste'] == 'others']
        v=1-accaste['pop'].sum()
        c=v+accaste.iloc[index_of_others[0], accaste.columns.get_loc('pop')]
        accaste.iloc[index_of_others[0], accaste.columns.get_loc('pop')]=c
        accaste=accaste[['clean caste','pop']]
        ac_caste_dict = accaste.set_index('clean caste')['pop'].to_dict()
        

        #5. Creating caste_map in raw
        ac['caste_map'] = np.where(ac['caste_clean'].isin(s3), ac['caste_clean'], 'others')


        ######################################################RETRO DICT FOR WEIGHING AND RETRO MAP COLUMN
        #1. finding party with more than 5 samples
        l= ac['mla_2018'].value_counts()
        l=l.reset_index(name='count')
        l=l[l['count']>5]
        s1=set(l['mla_2018'])

        #2. FINDING PARTY WITH MORE THAN 6% IN UNIVERSE
        acretro=retro[(retro['Key']=='2018 AE') & (retro['AC No']==aclist[i])]
        l= acretro[acretro['Vote Share Percentage']>6]
        s2=set(l['Party Name'])

        #3. Finsing intersection
        s3=s1.intersection(s2)
        s3.add('others')

        #4. Making retro dict for weighing
        acretro=retro[(retro['Key']=='2018 AE') & (retro['AC No']==aclist[i])]
        acretro=acretro[['Party Name','Vote Share Percentage']]
        acretro=acretro[acretro['Party Name'].isin(s3)].reset_index()
        acretro=acretro[['Party Name','Vote Share Percentage']]
        new_row = {'Party Name': 'others','Vote Share Percentage':100-acretro['Vote Share Percentage'].sum()}
        acretro.loc[len(acretro)] = new_row
        ac_retro_dict = acretro.set_index('Party Name')['Vote Share Percentage'].to_dict()
    

        #5. adding column in raw
        ac['party clean 2018'] = np.where(ac['mla_2018'].isin(s3), ac['mla_2018'], 'others')

        #################################################################################WEIGHING
        print("running ac no",aclist[i],"count",i)
        targets = {
        "Age bucket": {"18 to 24": 14.0, "25 to 35": 23.0, "35 to 50": 31.0, "51 & Above": 32.0},
        "gender": {"Male": 48, "Female": 52},
        "caste_clean": ac_caste_dict,
        "party clean 2018": ac_retro_dict

        }
        scheme = wp.scheme_from_dict(targets)

        ms_weighted = wp.weight_dataframe(
            df=ac,
            scheme=scheme,
            weight_column="wt_wpy_ac"
        )
        efficiency = wp.weighting_efficiency(ms_weighted["wt_wpy_ac"])

    except:
        print("ERROR OCCURED")
        ms_weighted=ac
        
        ###############################################################################MERGING
    if  i==0:
        ms_weighted_ac=ms_weighted
    else:
        ms_weighted_ac=pd.concat([ms_weighted_ac,ms_weighted],ignore_index=True)
    




In [187]:
ms_weighted_ac.to_excel('ms_weighted_ac.xlsx')

In [36]:
#Caste Universe dict creation
aclist=ms['ac_no'].unique()
shortlisted=caste[caste['ac_no'].isin(aclist)]
casteU=shortlisted.groupby([ 'clean caste'])
casteUdf=casteU.agg({'population':'sum'})
casteUdf['prop']=casteUdf['population']/casteUdf['population'].sum()
casteUdf=casteUdf[['prop']]
caste_dict = casteUdf['prop'].to_dict() #dictionery
caste_dict



#Universe weights
targets = {
    "Age bucket": {"18 to 24": 14.0, "25 to 35": 23.0, "35 to 50": 31.0, "51 & Above": 32.0},
    "gender": {"Male": 49, "Female": 51},
    "caste_clean": caste_dict,
    "party clean 2018": {"INC":42.0, "BJP": 42.1, "BSP":4.0, "others": 11.9 }

}
scheme = wp.scheme_from_dict(targets)

ms_weighted = wp.weight_dataframe(
    df=ms,
    scheme=scheme,
    weight_column="wt_wpy_U"
)
efficiency = wp.weighting_efficiency(df_weighted["wt_wpy_U"])

Raking achieved only partial convergence, please check the results to ensure that sufficient convergence was achieved.
No improvement was apparent after 197 iterations
